In [1]:
import keras
import csv
import os
import itertools
import scipy.io
import numpy as np
import tensorflow as tf
from keras.layers import *
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121
from keras import backend as k
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
ROOT_DIR = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath('__file__'))))
TRAIN_DIR = os.path.join(ROOT_DIR, 'data', 'train')
TEST_DIR =  os.path.join(ROOT_DIR, 'data', 'validation')

In [4]:
img_width, img_height = 300, 300
x_train, x_test, trainy, testy = [], [], [], []
batch_ = 0
batch_test = 0
batch_size = 32

In [ ]:
train_gen = ImageDataGenerator(rescale=1. / 255, zoom_range=0.2, horizontal_flip = True, vertical_flip = True)

In [ ]:
train_generator = train_gen.flow_from_directory(TRAIN_DIR, target_size=(img_width, img_height), 
                                                batch_size=batch_size, class_mode='categorical')

Found 152397 images belonging to 13 classes.


In [ ]:
test_gen = ImageDataGenerator(rescale=1. / 255, zoom_range=0.2, horizontal_flip = True, vertical_flip = True)

In [ ]:
test_generator = test_gen.flow_from_directory(TEST_DIR, target_size=(img_width, img_height), 
                                                batch_size=batch_size, class_mode='categorical')

Found 55388 images belonging to 13 classes.


In [ ]:
y_train = train_generator.classes
y_test = test_generator.classes

In [ ]:
input_ = Input(shape=(img_width, img_height, 3))

In [ ]:
def model_create(transfer_model):
    if transfer_model == 'InceptionV3':
        with tf.device('/gpu:1'):
            model_ = InceptionV3(input_tensor = input_, weights = 'imagenet', include_top = False)
        return Model(inputs = model_.input, outputs = model_.output)
    elif transfer_model == 'MobileNet':
        with tf.device('/gpu:1'):
            model_ = MobileNet(input_tensor = input_, weights = 'imagenet', include_top = False)
        return Model(inputs = model_.input, outputs = model_.output)
    elif transfer_model == 'VGG16':
        with tf.device('/gpu:1'):
            model_ = VGG16(input_tensor = input_, weights = 'imagenet', include_top = False)
        return Model(inputs = model_.input, outputs = model_.output)
    elif transfer_model == 'VGG19':
        with tf.device('/gpu:1'):
            model_ = VGG19(input_tensor = input_, weights = 'imagenet', include_top = False)
        return Model(inputs = model_.input, outputs = model_.output)
    elif transfer_model == 'Xception':
        with tf.device('/gpu:1'):
            model_ = Xception(input_tensor = input_, weights = 'imagenet', include_top = False)
        return Model(inputs = model_.input, outputs = model_.output)
    elif transfer_model == 'DenseNet121':
        with tf.device('/gpu:1'):
            model_ = DenseNet121(input_tensor = input_, weights = 'imagenet', include_top = False)
        return Model(inputs = model_.input, outputs = model_.output)

In [ ]:
def feature_extract(gen, model, y):
    return model.predict_generator(gen, int(len(y)/batch_size + 1))

In [ ]:
def save_features(file, feature):
    print('Saving {}'.format(file))
    scipy.io.savemat(file, {'x': feature[0], 'y': feature[1]}) 

In [ ]:
for model in ['InceptionV3', 'MobileNet', 'VGG16', 'VGG19', 'Xception', 'DenseNet121']:
    print(model)
    filename = '{}/{}.mat'.format(ROOT_DIR, model)
    transfer_model = model_create(model)
    print('Extracting features from train data for model {}'.format(model))
    features = feature_extract(train_generator, transfer_model, y_train)
    print('Extracting features from test data for model {}'.format(model))
    features_test = feature_extract(test_generator, transfer_model, y_test)
    
    for feature in [(features, y_train), (features_test, y_test)]:
        save_features(filename, feature)

InceptionV3
Extracting features from train data for model InceptionV3
